## Axion log

#### Transacciones PDF 

In [106]:
pip install pymupdf pdfplumber tabula-py camelot-py[cv] pytesseract Pillow pandas camelot

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [107]:
# Lectura de pdf
import fitz  
import pdfplumber
import tabula
import pytesseract
from PIL import Image
import pandas as pd
import io
import re
import pandas as pd
import unicodedata

In [108]:


class PDFExtractorAdaptativo:
    def __init__(self):
        self.metodos_texto = ['pdfplumber', 'pymupdf', 'ocr']
        self.metodos_tablas = ['tabula', 'pdfplumber']  # Quitamos 'camelot'
    
    def detectar_tipo_contenido(self, archivo_pdf):
        """Detecta qué tipo de contenido tiene el PDF"""
        try:
            doc = fitz.open(archivo_pdf)
            primera_pagina = doc[0]
            
            texto = primera_pagina.get_text().strip()
            imagenes = primera_pagina.get_images()
            
            with pdfplumber.open(archivo_pdf) as pdf:
                page = pdf.pages[0]
                tablas_detectadas = page.find_tables()
            
            resultado = {
                'tiene_texto': len(texto) > 50,
                'tiene_imagenes': len(imagenes) > 0,
                'posibles_tablas': len(tablas_detectadas) > 0,
                'es_escaneado': len(texto) < 50 and len(imagenes) > 0,
                'calidad_texto': 'alta' if len(texto) > 200 else 'baja'
            }
            
            doc.close()
            return resultado
            
        except Exception as e:
            return {'error': str(e)}
    
    def extraer_texto_robusto(self, archivo_pdf):
        """Extrae texto probando diferentes métodos"""
        metodos_resultados = {}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                texto = ""
                for page in pdf.pages:
                    texto += page.extract_text() + "\n"
                metodos_resultados['pdfplumber'] = texto
        except Exception as e:
            metodos_resultados['pdfplumber'] = f"Error: {e}"
        
        # pymupdf
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page in doc:
                texto += page.get_text() + "\n"
            doc.close()
            metodos_resultados['pymupdf'] = texto
        except Exception as e:
            metodos_resultados['pymupdf'] = f"Error: {e}"
        
        # OCR
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                pix = page.get_pixmap()
                img_data = pix.tobytes("png")
                img = Image.open(io.BytesIO(img_data))
                texto += pytesseract.image_to_string(img, lang='spa') + "\n"
            doc.close()
            metodos_resultados['ocr'] = texto
        except Exception as e:
            metodos_resultados['ocr'] = f"Error: {e}"
        
        # Seleccionar mejor resultado
        mejor_resultado = ""
        max_longitud = 0
        for metodo, resultado in metodos_resultados.items():
            if not resultado.startswith("Error") and len(resultado) > max_longitud:
                mejor_resultado = resultado
                max_longitud = len(resultado)
        
        return {
            'texto_final': mejor_resultado,
            'metodos_intentados': metodos_resultados,
            'metodo_exitoso': max([k for k, v in metodos_resultados.items() 
                                 if not v.startswith("Error") and len(v) == max_longitud], 
                                default="ninguno")
        }
    
    def extraer_tablas_robusto(self, archivo_pdf):
        """Extrae tablas probando diferentes métodos (sin camelot)"""
        resultados_tablas = {}
        
        # tabula-py
        try:
            tablas = tabula.read_pdf(archivo_pdf, pages='all', multiple_tables=True)
            resultados_tablas['tabula'] = {
                'tablas': tablas,
                'cantidad': len(tablas),
                'exito': True
            }
        except Exception as e:
            resultados_tablas['tabula'] = {'error': str(e), 'exito': False}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                todas_tablas = []
                for page in pdf.pages:
                    tablas_pagina = page.extract_tables()
                    for tabla in tablas_pagina:
                        df = pd.DataFrame(tabla[1:], columns=tabla[0])
                        todas_tablas.append(df)
                
                resultados_tablas['pdfplumber'] = {
                    'tablas': todas_tablas,
                    'cantidad': len(todas_tablas),
                    'exito': True
                }
        except Exception as e:
            resultados_tablas['pdfplumber'] = {'error': str(e), 'exito': False}
        
        return resultados_tablas
    
    def procesar_pdf_completo(self, archivo_pdf):
        """Proceso completo adaptativo"""
        print(f"Analizando: {archivo_pdf}")
        
        info_contenido = self.detectar_tipo_contenido(archivo_pdf)
        print(f"Análisis de contenido: {info_contenido}")
        
        resultado_final = {
            'archivo': archivo_pdf,
            'analisis_contenido': info_contenido,
            'texto': None,
            'tablas': None
        }
        
        if info_contenido.get('tiene_texto') or info_contenido.get('es_escaneado'):
            print("Extrayendo texto...")
            resultado_texto = self.extraer_texto_robusto(archivo_pdf)
            resultado_final['texto'] = resultado_texto
        
        if info_contenido.get('posibles_tablas'):
            print("Extrayendo tablas...")
            resultado_tablas = self.extraer_tablas_robusto(archivo_pdf)
            resultado_final['tablas'] = resultado_tablas
        
        return resultado_final

# Ejemplo de uso
if __name__ == "__main__":
    extractor = PDFExtractorAdaptativo()
    resultado = extractor.procesar_pdf_completo("resOurces/AXION_LOG.pdf")


Analizando: resOurces/AXION_LOG.pdf
Análisis de contenido: {'tiene_texto': True, 'tiene_imagenes': True, 'posibles_tablas': True, 'es_escaneado': False, 'calidad_texto': 'alta'}
Extrayendo texto...
Extrayendo tablas...


In [109]:
def convertir_numero(valor):
    # Eliminar espacios
    valor = valor.strip()
    # Caso coma como decimal
    if ',' in valor:
        valor = valor.replace('.', '').replace(',', '.')
    return float(valor)

In [110]:
def limpiar_columnas(df):
    """
    Limpia los nombres de las columnas de un DataFrame:
    - Elimina espacios al inicio y final
    - Convierte a minúsculas
    - Quita tildes y acentos
    - Reemplaza espacios y caracteres especiales por guiones bajos
    - Elimina caracteres no alfanuméricos excepto guiones bajos
    - Evita guiones bajos duplicados
    """
    def normalizar_texto(texto):
        # Quitar tildes y acentos usando unicodedata
        texto_sin_acentos = unicodedata.normalize('NFD', texto)
        texto_sin_acentos = ''.join(c for c in texto_sin_acentos if unicodedata.category(c) != 'Mn')
        return texto_sin_acentos
    
    df.columns = (
        df.columns
        .str.strip()                                    # Quitar espacios al inicio y final
        .str.lower()                                    # Convertir a minúsculas
        .map(normalizar_texto)                          # Quitar tildes y acentos
        .str.replace(r'[\s\-]+', '_', regex=True)       # Espacios y guiones por guiones bajos
        .str.replace(r'[^\w]', '_', regex=True)         # Caracteres especiales por guiones bajos
        .str.replace(r'_+', '_', regex=True)            # Múltiples guiones bajos por uno solo
        .str.strip('_')                                 # Quitar guiones bajos al inicio y final
    )
    return df

In [111]:
#extraer del diccionario final tablas y texto
textos = resultado.get('texto', {}).get('texto_final', '')
print(type(textos))

#Escritura .txt
with open("AXION_LOG.txt", "w", encoding="utf-8") as f:
    f.write(textos)

<class 'str'>


In [112]:
texto = textos

In [113]:
# === 1. Extraer encabezado (solo el primero)
encabezado = {}
encabezado['Fecha Orden'] = re.search(r'Fecha de la Orden\s+(\d{2}/\d{2}/\d{4})', texto).group(1)
encabezado['Pedido a'] = re.search(r'Pedido a\s+(.*?)\n', texto).group(1).strip()
encabezado['Dirección'] = re.search(r'CALLE [^\n]+', texto).group(0).strip()
encabezado['Emitido en'] = re.search(r'Emitido en\s+(.*?)\n', texto).group(1).strip()
encabezado['Nro Nota Pedido'] = re.search(r'Nro de Nota de Pedido\s+(\d+)', texto).group(1)
encabezado['Condiciones Pago'] = re.search(r'Condiciones de Pago:\s+(.*?)\s+Ultima Fecha', texto).group(1).strip()
encabezado['Última Fecha Entrega'] = re.search(r'Ultima Fecha de Entrega:\s+(\d{2}/\d{2}/\d{4})', texto).group(1)
encabezado['Destino'] = re.search(r'Destino:\s+([^\n]+)', texto).group(1).strip()
encabezado

{'Fecha Orden': '29/04/2025',
 'Pedido a': 'CROC SAS',
 'Dirección': 'CALLE 69 # 19 -36 BOGOTA',
 'Emitido en': 'TENJO',
 'Nro Nota Pedido': '655664',
 'Condiciones Pago': '45 Días Fecha Factura',
 'Última Fecha Entrega': '08/05/2025',
 'Destino': '0101 Axis Log. SAS-BOG.'}

### Escoger Columnas -Ordenar dataframe
1. Cliente
2. Direccion Cliente
3. Destino
4. Numero de Pedido
5. Fecha de Orden
6. Fecha de Entrega
7. Id Falso (Para Cruce de Orden de compra )


In [114]:
df_encabezado = pd.DataFrame([encabezado])
df_encabezado = limpiar_columnas(df_encabezado)

In [115]:
df_encabezado

,fecha_orden,pedido_a,direccion,emitido_en,nro_nota_pedido,condiciones_pago,ultima_fecha_entrega,destino
0,29/04/2025,CROC SAS,CALLE 69 # 19 -36 BOGOTA,TENJO,655664,45 Días Fecha Factura,08/05/2025,0101 Axis Log. SAS-BOG.


In [116]:
#Chose Columns
#create false columns
df_axionlog = df_encabezado.copy()
df_axionlog["CLIENTE"] = 'Axionlog'
df_axionlog["id"] = 1
df_axionlog["direccion"] = df_axionlog["destino"]

#renombrar NUMERO_DE_PEDIDO, FECHA_DE_ORDEN, FECHA_DE_ENTREGA, CREAR_ID_FALSO
df_axionlog.rename(columns={
    'nro_nota_pedido': 'NUMERO_DE_PEDIDO',
    'fecha_orden': 'FECHA_DE_ORDEN',
    'ultima_fecha_entrega': 'FECHA_DE_ENTREGA',
    'direccion': 'DIRECCION',
    'destino': 'DESTINO'
}, inplace=True)

#seleccionar columnas
df_axionlog = df_axionlog[['id', 'CLIENTE', 'DIRECCION', 'DESTINO', 'NUMERO_DE_PEDIDO', 'FECHA_DE_ORDEN', 'FECHA_DE_ENTREGA']]

In [117]:
df_axionlog

,id,CLIENTE,DIRECCION,DESTINO,NUMERO_DE_PEDIDO,FECHA_DE_ORDEN,FECHA_DE_ENTREGA
0,1,Axionlog,0101 Axis Log. SAS-BOG.,0101 Axis Log. SAS-BOG.,655664,29/04/2025,08/05/2025


#### Productos 

In [118]:
# Extraer líneas donde comienzan con patrón tipo: "1.000\n0\n496\nDESCRIPCION..."
patron = re.compile(
    r"(\d+\.\d+)\s+0\s+(\d+)\s+(.*?)\s+([\d.,]+)\s+(\w+)\s+(\d{2}/\d{2}/\d{4})\s+([\d.,]+)\s+([\d.,]+)",
    re.DOTALL
)

matches = patron.findall(texto)

# Crear DataFrame
df = pd.DataFrame(matches, columns=[
    'Linea', 'Item', 'Descripción', 'Cantidad',
    'Unidad Medida', 'Fecha Entrega', 'Costo Unitario', 'Importe COP'
])

# Convertir columnas numéricas
df['Cantidad'] = df['Cantidad'].apply(convertir_numero)
df['Costo Unitario'] = df['Costo Unitario'].apply(convertir_numero)
df['Importe COP'] = df['Importe COP'].apply(convertir_numero)

In [119]:
matches

[('1.000',
  '496',
  'DON CHICHARRON 35 GR',
  '660.0000',
  'UN',
  '08/05/2025',
  '3.166,19',
  '2.089.685,40'),
 ('2.000',
  '3892',
  'CABANO BAKANO 24GR x 10 UN',
  '120.0000',
  'PQ',
  '08/05/2025',
  '23.950,00',
  '2.874.000,00'),
 ('3.000',
  '2263',
  'DON CHICHARRON ESPIRAL 100GR',
  '1,800.0000',
  'UN',
  '08/05/2025',
  '8.460,48',
  '15.228.864,00'),
 ('4.000',
  '3083',
  'DON CHICHARRONESPIRAL CH 100GR',
  '1,500.0000',
  'UN',
  '08/05/2025',
  '1.868,57',
  '2.802.855,00'),
 ('1.000',
  '496',
  'DON CHICHARRON 35 GR',
  '660.0000',
  'UN',
  '08/05/2025',
  '3.166,19',
  '2.089.685,40'),
 ('2.000',
  '3892',
  'CABANO BAKANO 24GR x 10 UN',
  '120.0000',
  'PQ',
  '08/05/2025',
  '23.950,00',
  '2.874.000,00'),
 ('3.000',
  '2263',
  'DON CHICHARRON ESPIRAL 100GR',
  '1,800.0000',
  'UN',
  '08/05/2025',
  '8.460,48',
  '15.228.864,00'),
 ('4.000',
  '3083',
  'DON CHICHARRONESPIRAL CH 100GR',
  '1,500.0000',
  'UN',
  '08/05/2025',
  '1.868,57',
  '2.802.855,00')

In [120]:
df = limpiar_columnas(df)

#### Escoger Columnas Productos 
1. Numero de item
2. descripcion
3. Cantidad
4. Unidad de medida
5. Fecha de entrega
6. Costo Unitario
7. importe
8. id falso (Unir para realizar insersion de datos)

In [121]:
#rename Columns 
df_axionlog_productos = df.copy()

#rename columns
df_axionlog_productos.rename(columns={
    'item': 'NUMERO_DE_ITEM',
    'descripcion': 'DESCRIPCION',
    'cantidad': 'CANTIDAD',
    'unidad_medida': 'UNIDAD_DE_MEDIDA',
    'fecha_entrega': 'FECHA_ENTREGA',
    'costo_unitario': 'COSTO_UNITARIO',
    'importe_cop': 'IMPORTE'
}, inplace=True)

#create false columns
df_axionlog_productos["id"] = 1

#Select columns
df_axionlog_productos = df_axionlog_productos[['id', 'NUMERO_DE_ITEM', 'DESCRIPCION', 'CANTIDAD', 'UNIDAD_DE_MEDIDA', 'COSTO_UNITARIO', 'IMPORTE']]

#### JOIN

In [122]:
df_merge_axionlog = pd.merge(df_axionlog, df_axionlog_productos, on='id', how='inner')

#drop duplicates
df_merge_axionlog = df_merge_axionlog.drop_duplicates()

In [124]:
# Mostrar el DataFrame
df_merge_axionlog

,id,CLIENTE,DIRECCION,DESTINO,NUMERO_DE_PEDIDO,FECHA_DE_ORDEN,FECHA_DE_ENTREGA,NUMERO_DE_ITEM,DESCRIPCION,CANTIDAD,UNIDAD_DE_MEDIDA,COSTO_UNITARIO,IMPORTE
0,1,Axionlog,0101 Axis Log. SAS-BOG.,0101 Axis Log. SAS-BOG.,655664,29/04/2025,08/05/2025,496,DON CHICHARRON 35 GR,660.0,UN,3166.19,2089685.4
1,1,Axionlog,0101 Axis Log. SAS-BOG.,0101 Axis Log. SAS-BOG.,655664,29/04/2025,08/05/2025,3892,CABANO BAKANO 24GR x 10 UN,120.0,PQ,23950.00,2874000.0
2,1,Axionlog,0101 Axis Log. SAS-BOG.,0101 Axis Log. SAS-BOG.,655664,29/04/2025,08/05/2025,2263,DON CHICHARRON ESPIRAL 100GR,1.8,UN,8460.48,15228864.0
3,1,Axionlog,0101 Axis Log. SAS-BOG.,0101 Axis Log. SAS-BOG.,655664,29/04/2025,08/05/2025,3083,DON CHICHARRONESPIRAL CH 100GR,1.5,UN,1868.57,2802855.0


In [ ]:
#120 * 10 1200 ->Kabano ->Cruce de Numero de Item ->Homologar productos, Cambiar cantidades ->Formato cantidades , ciudad adicional Bogota
# Probar otras ordenes 